In [ ]:
from check.find_ex_reactions import *
from check.set_metabolite_bounds import *
from check.find_required_rxns import *
from check.check_rxn_flux import *
from cobra import *

In [ ]:
precursorMets = '../pre_processing/dataset/key_metabolites_RECON1.txt'

### Test find_ex_reactions.py

In [ ]:
print('## Testing find_ex_reactions...')

model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')

try:

    exRxns = find_ex_rxns(model)
    
    if len(exRxns) == 430:

        print('Function returns the [expected] result: %s exchange & demand reactions' %len(exRxns))

    else:

        print('Function returns the [unexpected] result: %s exchange & demand reactions' %len(exRxns))

    
    print('PASS...', 'Function find_ex_rxns ran without error')

except (RuntimeError, TypeError, NameError) as inst:

    print('FAIL...', 'Function find_ex_rxns was terminated with the error:')
    print(type(inst))
    print(inst.args)

### Test find_organic_ex_rxns.py

In [ ]:
display('## Testing find_organic_ex_rxns...');

model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')

try:
    organicExRxns = find_organic_ex_rxns(model, exRxns)
    
    if len(organicExRxns) == 406:
        
        print('Function returns the [expected] result: %s organic reactions' %len(organicExRxns))
    
    else:
        
        print('Function returns the [unexpected] result: %s organic reactions' %len(organicExRxns))

    
    print('PASS...', 'Function find_organic_ex_rxns ran without error')
    
except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function find_organic_ex_rxns was terminated with the error:')
    print(type(inst))
    print(inst.args)

### Test set_metabolite_bounds.py

In [ ]:
display('## Testing set_organic_met_bounds...');
model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')
try:
    testModel = set_organic_met_bounds(model, exRxns)
    print('PASS...', 'Function set_organic_met_bounds ran without error')
    
except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function set_organic_met_bounds was terminated with the error:')
    print(type(inst))
    print(inst.args)
    
# check all bounds were set correctly
print("## Testing output of set_organic_met_bounds...")
# check if all organic reactions are in model
for r in organicExRxns:
    if testModel.reactions.get_by_id(r).lower_bound != 0:
        print("Bound for [%s] not set correctly" %r)

### Test find_required_rxns.py

In [ ]:
display('## Testing find_required_rxns...');
model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')
try:
    testModel, requiredRxns = find_required_rxns(model, '../pre_processing/dataset/key_metabolites_RECON1.txt')
    print('PASS...', 'Function find_required_rxns ran without error')
    
except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function find_required_rxns was terminated with the error:')
    print(type(inst))
    print(inst.args)

### Test check_rxn_flux.py

In [ ]:
print('## Checking for success on preceding tests...')
model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')
try:
    
    metList = precursorMets
    exRxns = find_ex_rxns(model)
    testModel = set_organic_met_bounds(model, exRxns)
    
    
    testModel, requiredRxns = find_required_rxns(model, metList)

    # Allow uptake of glucose and CO2
    testModel.reactions.get_by_id("EX_glc__D_e").lower_bound = -5 
    testModel.reactions.get_by_id("EX_co2_e").lower_bound = -1000 

    
except(RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Cannot test check_rxn_flux because one of the preceding functions returned the error:')
    print(type(inst))
    print(inst.args)
          
# Test whether Recon1 passes functional check 
print('## Testing check_rxn_flux with functional input...')   

try:
    inactiveRequired = check_rxn_flux(testModel, requiredRxns)
    print('PASS...','Function check_rxn_flux ran without error')
    
except(RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function check_rxn_flux was terminated with the error:')
    print(type(inst))
    print(inst.args)
    
# Test if Recon 1 passes check when you 'break' it (turn off glucose)
print('## Testing check_rxn_flux with glucose uptake off...')
try:
    with testModel:
        testModel.reactions.get_by_id("EX_glc__D_e").lower_bound == 0
        inactiveRequired = check_rxn_flux(testModel, requiredRxns)
        print('PASS...', 'Function check_rxn_flux ran without error')

except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function check_rxn_flux was terminated with the error:')
    print(type(inst))
    print(inst.args)
    

# Test if Recon 1 passes check when you 'break' it (output blocked)    
print('## Testing check_rxn_flux with one output blocked...')
try:
    with testModel:
        testModel.reactions.get_by_id("DM_3pg_c").upper_bound == 0
        inactiveRequired = check_rxn_flux(testModel, requiredRxns)
        print('PASS...', 'Function check_rxn_flux ran without error')

except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function check_rxn_flux was terminated with the error:')
    print(type(inst))
    print(inst.args)

### Test check_model_function.py

In [ ]:
from check.check_model_function import *
# test with default inputs
print('## Testing check_model_function with default inputs...')
model = io.read_sbml_model('../pre_processing/dataset/recon1_with_BOF_and_VBOF.xml')

try:
    
    genericStatus = check_model_function(model,'requiredMets', precursorMets)
    print('PASS...', 'Function check_model_function ran without error')
    
    if genericStatus:
        print('PASS...', 
            'Check for functionality returns expected result')
    else:
        print('FAIL...', 
            'Check for functionality returns unexpected result')
        
except (RuntimeError, TypeError, NameError) as inst:
    
    print('FAIL...', 'Function check_model_function was terminated with the error:')
    print(type(inst))
    print(inst.args)
    